<a href="https://colab.research.google.com/github/nmuzyka2004/Proyecto-Jupiter_Mercadona/blob/main/Web_scraping_Dia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Instalar selenium
!pip install selenium
!pip install fake_useragent

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [16]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')

In [17]:
ua = UserAgent()
chrome_options.add_argument(f'user-agent={ua.random}')

In [18]:
# inicializamos el WebDriver:
driver = webdriver.Chrome(options=chrome_options)

In [19]:
driver.get("https://www.dia.es")
driver.set_window_size(1920, 1200)
time.sleep(5)
driver.save_screenshot('inicio.png')

True

In [20]:
# Simulamos un clic en el elemento "Acceptar los cookies"
boton = driver.find_element(By.ID, "onetrust-accept-btn-handler")
boton.click()
time.sleep(3)
driver.save_screenshot('cookie_acceptar.png')

True

In [21]:
# Simulamos un clic en el elemento "Productos" para abrir las categorias
boton = driver.find_element(By.CSS_SELECTOR, "button[data-test-id='desktop-category-button']")
boton.click()
time.sleep(3)
# driver.save_screenshot('dia_abrir_categorias.png')

In [22]:
# Creamos una lista de categorias
soup = bs(driver.page_source, 'html.parser')
categorias_productos = soup.select("a[data-test-id='category-item-link']")

categorias = []
for categoria in categorias_productos:
    nombre = categoria.select_one("span.category-item__title")
    nombre_categoria = nombre.get_text(strip=True)
    local_url = categoria.get("href")
    categorias.append({"name": nombre_categoria, "url": local_url})

print(f"Tenemos {len(categorias)} categorias de productos")

Tenemos 24 categorias de productos


In [23]:
# Creamos una lista de subcategorias
subcategorias = []
base_url = "https://www.dia.es"

for categoria in categorias:
    name = categoria.get("name")
    local_url = categoria.get("url")
    full_url = base_url + local_url

    driver.get(full_url)
    WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[data-test-id='sub-category-item-link']")))

    soup = bs(driver.page_source, "html.parser")

    subcategorias_productos = soup.select("a[data-test-id='sub-category-item-link']")

    for subcategoria in subcategorias_productos:
        nombre = subcategoria.select_one("span[data-test-id='sub-category-item-title']")
        subcategoria_nombre = nombre.get_text(strip=True)
        subcategoria_local_url = subcategoria.get("href")
        subcategorias.append({
            "categoria": name,
            "subcategoria": subcategoria_nombre,
            "url": base_url + subcategoria_local_url})

# Guardamos los resultados en .csv
df_subcategorias = pd.DataFrame(subcategorias)
df_subcategorias.to_csv("subcategorias_dia.csv", index=False, encoding="utf-8-sig")

In [24]:
df_subcategorias.head()

,categoria,subcategoria,url
0,Freidora de aire - Airfryer,Todo freidora de aire - airfryer,https://www.dia.es/freidora-de-aire-airfryer/c...
1,Freidora de aire - Airfryer,Patatas Airfryer,https://www.dia.es/freidora-de-aire-airfryer/p...
2,Freidora de aire - Airfryer,Rebozados Airfryer,https://www.dia.es/freidora-de-aire-airfryer/r...
3,Freidora de aire - Airfryer,Verduras Airfryer,https://www.dia.es/freidora-de-aire-airfryer/v...
4,Freidora de aire - Airfryer,Pescados y mariscos Airfryer,https://www.dia.es/freidora-de-aire-airfryer/p...


In [25]:
df_subcategorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   categoria     213 non-null    object
 1   subcategoria  213 non-null    object
 2   url           213 non-null    object
dtypes: object(3)
memory usage: 5.1+ KB


In [26]:
productos = []

# Sacamos datos de todos los productos en cada subcategoría
for subcategoria in subcategorias:
    url = subcategoria.get("url")
    categoria = subcategoria.get("categoria")
    subcategory = subcategoria.get("subcategoria")

    # Eliminamos subcategorias acumulativas (Todo de {nombre de categoria})
    if subcategory.strip().lower().startswith(("todo", "todos")):
        continue

    # Para controlar el proceso:
    print(f"Estamos en subcategoria: {subcategory}")

    driver.get(url)
    WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CSS_SELECTOR, "li[data-test-id='product-card-list-item']")))

    # scroll
    # Vamos a simular acciones humanas
    actions = ActionChains(driver)

    # Aquí simulamos el comportamiento del usuario al bajar con Page Down
    previous_count = 0
    same_count_repeats = 0

    for _ in range(30):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)
        elements = driver.find_elements(By.CSS_SELECTOR, "li[data-test-id='product-card-list-item']")
        current_count = len(elements)

        if current_count == previous_count:
            same_count_repeats += 1
            if same_count_repeats >= 3:
                break
        else:
             same_count_repeats = 0
        previous_count = current_count

    time.sleep(2)
    # Screenshot de control
    # driver.save_screenshot(f"screenshot_{categoria}_{subcategory}.png")

    # tarjetas de productos
    elements = driver.find_elements(By.CSS_SELECTOR, "li[data-test-id='product-card-list-item']")

    for i, element in enumerate(elements):
        html = element.get_attribute("outerHTML")
        card = bs(html, "html.parser")
        tarjeta = card.select_one("div.search-product-card")

        # sacamos nombre de producto
        name_tag = tarjeta.select_one("p.search-product-card__product-name")
        name = name_tag.get_text(strip=True)

        # sacamos descripcion de producto
        description = None
        match = re.search(r"(bolsa|bandeja|pack|paquete|tarro|botella|caja|unidad|sobre|brick|lata|barqueta|tarrina|\d+\s*(?:g|kg|ml|l))", name, re.IGNORECASE)
        if match:
            description = match.group(0).strip()

        # sacamos el precio de producto
        price_tag = tarjeta.select_one("p[data-test-id='search-product-card-unit-price']")
        price = float(price_tag.get_text(strip=True).replace("€", "").replace(",", ".").replace("\xa0", ""))

        # sacamos reference_price y reference_unit
        reference_price = None
        reference_unit = None

        reference_tag = tarjeta.select_one("p[data-test-id='search-product-card-kilo-price']")
        reference_text = reference_tag.get_text(strip=True).replace("€", "").replace(",", ".").replace("(", "").replace(")", "").replace("\xa0", "")
        parts = reference_text.split("/")
        if len(parts) == 2:
            reference_price = float(parts[0].strip())
            reference_unit = parts[1].strip()

        # sacamos id de producto
        product_id = tarjeta.get("object_id")

        # fecha de hoy
        insert_date = datetime.now().strftime("%Y-%m-%d")

        # añadimos los datos de producto a la lista
        productos.append({
            "name": name,
            "description": description,
            "price": price,
            "reference_price": reference_price,
            "reference_unit": reference_unit,
            "product_id": product_id,
            "insert_date": insert_date,
            "category": categoria,
            "subcategory": subcategory
                })

    print(f"Productos encontrados: {len(elements)}")

# Paramos Driver
driver.quit()
# Guardamos los datos en .csv
df = pd.DataFrame(productos)
df.to_csv("productos_dia.csv", index=False, encoding="utf-8-sig")
print(f"Productos encontrados en total: {len(df)}")

Estamos en subcategoria: Patatas Airfryer
Productos encontrados: 6
Estamos en subcategoria: Rebozados Airfryer
Productos encontrados: 22
Estamos en subcategoria: Verduras Airfryer
Productos encontrados: 16
Estamos en subcategoria: Pescados y mariscos Airfryer
Productos encontrados: 33
Estamos en subcategoria: Carne Airfryer
Productos encontrados: 9
Estamos en subcategoria: Comida preparada Airfryer
Productos encontrados: 18
Estamos en subcategoria: Accesorios Airfryer
Productos encontrados: 4
Estamos en subcategoria: Jamón cocido, lacón, fiambres y mortadela
Productos encontrados: 59
Estamos en subcategoria: Jamón curado y paleta
Productos encontrados: 17
Estamos en subcategoria: Lomo, chorizo, fuet, salchichón
Productos encontrados: 41
Estamos en subcategoria: Queso curado, semicurado y tierno
Productos encontrados: 63
Estamos en subcategoria: Queso fresco
Productos encontrados: 23
Estamos en subcategoria: Queso azul y roquefort
Productos encontrados: 5
Estamos en subcategoria: Quesos

In [27]:
df = pd.read_csv('/content/productos_dia.csv')
df.head(20)

,name,description,price,reference_price,reference_unit,product_id,insert_date,category,subcategory
0,Patatas gajo Patatas Unidas Dia bolsa 750 g,bolsa,1.80,2.40,KILO,262685,2025-05-23,Freidora de aire - Airfryer,Patatas Airfryer
1,Patatas prefritas Patatas Unidas Dia bolsa 1 Kg,bolsa,1.85,1.85,KILO,262861,2025-05-23,Freidora de aire - Airfryer,Patatas Airfryer
2,Patatas prefritas Patatas Unidas Dia bolsa 2 Kg,bolsa,3.10,1.55,KILO,280262,2025-05-23,Freidora de aire - Airfryer,Patatas Airfryer
3,Patatas finas prefritas Patatas Unidas Dia bol...,bolsa,1.93,1.93,KILO,262863,2025-05-23,Freidora de aire - Airfryer,Patatas Airfryer
4,Patatas gajo barbacoa McCain bolsa 750 g,bolsa,3.59,4.79,KILO,250786,2025-05-23,Freidora de aire - Airfryer,Patatas Airfryer
5,Tortilla de patatas con cebolla La cocina de s...,caja,6.99,11.65,KILO,305263,2025-05-23,Freidora de aire - Airfryer,Patatas Airfryer
6,Mini empanadillas de atún Al Punto Dia bolsa 4...,bolsa,1.57,3.93,KILO,1413,2025-05-23,Freidora de aire - Airfryer,Rebozados Airfryer
7,Varitas de pescado MSC Findus caja 250 g,caja,2.99,11.96,KILO,NaN,2025-05-23,Freidora de aire - Airfryer,Rebozados Airfryer
8,Rabas empanadas Mari Marinera de Dia bolsa 400 g,bolsa,3.39,8.48,KILO,41098,2025-05-23,Freidora de aire - Airfryer,Rebozados Airfryer
9,Croquetas de cocido Al Punto Dia bolsa 350 g,bolsa,2.00,5.71,KILO,304800,2025-05-23,Freidora de aire - Airfryer,Rebozados Airfryer


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6218 entries, 0 to 6217
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             6218 non-null   object 
 1   description      6212 non-null   object 
 2   price            6218 non-null   float64
 3   reference_price  6218 non-null   float64
 4   reference_unit   6218 non-null   object 
 5   product_id       6088 non-null   object 
 6   insert_date      6218 non-null   object 
 7   category         6218 non-null   object 
 8   subcategory      6218 non-null   object 
dtypes: float64(2), object(7)
memory usage: 437.3+ KB
